In [1]:
import numpy as np
import random as rn
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
def probability_of_acceptance(T, c_v, n_v):
    return np.exp(-abs(n_v - c_v)/T)

### Zapisywanie GIF-ów
Drobna modyfikacja funkcji w celu możliwości zapisywania gifów oraz optymalizacji obliczania energii

In [3]:
def optimized_sim_annealing(s, max_iters, T, alfa, swap_and_update_loss_func, swap_func, change_temp_func,
                            loss_func, initial_loss_func, probability_of_acceptance_func=probability_of_acceptance, save=False, filename=None, duration=40):
    best = float('inf')
    best_result = None
    vals = []
    curr_val = initial_loss_func(s, loss_func)
    step = max_iters//200
    frames = []
    if save and filename is None:
        print("To save, choose a filename")
        return None, None, None

    for k in range(max_iters):
        T = change_temp_func(T, alfa, k)

        vals.append(curr_val)

        new_s, new_val = swap_and_update_loss_func(s, curr_val, swap_func, loss_func)
        
        if save and k % step == 0:
            array = (1 - s) * 255  # Zamieniamy: 0 -> 255 (biały), 1 -> 0 (czarny)
            img = Image.fromarray(array.astype(np.uint8), mode='L')  # L = grayscale
            img = img.resize((500, 500), resample=Image.NEAREST)
            frames.append(img)

        if new_val < curr_val or rn.uniform(0, 1) <= probability_of_acceptance_func(T, curr_val, new_val):
            s = new_s
            curr_val = new_val
            if new_val < best:
                best = new_val
                best_result = new_s.copy()
    if frames:
        frames[0].save(
            filename,
            save_all=True,
            append_images=frames[1:],
            duration=duration,
            loop=0,
            optimize=False
        )
    return best, vals, best_result


In [4]:
def generate_image(n, delta):
    image = np.random.binomial(1, delta, size=(n, n))
    return image

Funkcje pomocnicze

In [5]:
def calculate_all_energy(image, energy_function):
    energy = 0
    for x in range(len(image)):
        for y in range(len(image)):
            energy += energy_function(image, x, y)
    return energy


In [6]:
def img_swap_and_update_energy(image, energy, swap_function, energy_function):
    new_image = image.copy()
    old_pts, new_pts = swap_function(image)

    for i in range(len(old_pts)):
        x1, y1 = old_pts[i]
        x2, y2 = new_pts[i]
        val1 = new_image[x1, y1].copy()
        val2 = new_image[x2, y2].copy()
        if val1 == val2:
            continue
        energy -= 2 * energy_function(image, x1, y1)
        energy -= 2 * energy_function(image, x2, y2)
        new_image[x1, y1] = val2
        new_image[x2, y2] = val1
    return new_image, energy


Funkcje zamieniające piksele

In [7]:
def img_swap_random(image):
    new_pts = []
    old_pts = []
    for _ in range(4):
        p1 = np.random.randint(0, len(image) - 1, size=2)
        p2 = np.random.randint(0, len(image) - 1, size=2)
        new_pts.append(p1)
        old_pts.append(p2)
    return old_pts, new_pts

In [8]:
def img_swap_adjacent(image):
    new_pts = []
    old_pts = []
    for _ in range(4):
        p1 = np.random.randint(1, len(image) - 2, size=2)
        move = np.random.randint(-1, 1, size=2)
        p2 = [p1[0] + move[0], p1[1] + move[1]]
        new_pts.append(p1)
        old_pts.append(p2)
    return old_pts, new_pts

Różne funkcje obliczania energii

In [10]:
def ising(image, x, y, adjacent):
    energy = 0
    for m_x, m_y in adjacent:
        if 0 < x + m_x < len(image) and 0 < y + m_y < len(image):
            energy += -1 if image[x + m_x, y + m_y] == image[x, y] else 1
    return energy

In [11]:
def energy_ising_8(image, x, y):
    adjacent = [(1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1), (0, 1)]
    return ising(image, x, y, adjacent)


In [12]:
def energy_ising_16(image, x, y):
    adjacent = [(-2, 2), (-1, 2), (0, 2), (1, 2), (2, 2), (-2, -2), (-1, -2), (0, -2), (1, -2), (2, -2), (-2, -1),
                (-2, 0), (-2, 1), (2, -1), (2, 0), (2, 1)]
    return ising(image, x, y, adjacent)

In [13]:
def energy_push_and_pull(image, x, y):
    energy = -energy_ising_8(image, x, y)
    energy += energy_ising_16(image, x, y)
    return energy

In [48]:
def energy_horiz(image,x,y):
    vertical = [(0, i) for i in range(-2, 3)]
    horizontal = [(i, 0) for i in range(-2, 3)]
    energy = -ising(image, x, y, horizontal)
    energy += ising(image, x, y, vertical)
    return energy

In [49]:
def energy_cross(image,x,y):
    cross = [(i,i) for i in range(-5, 6)] + [(i,-i) for i in range(-5, 6)]
    return ising(image, x, y, cross)

In [53]:
def energy_vert_horiz(image,x,y):
    vertical = [(0, i) for i in range(-3, 4)]
    horizontal = [(i, 0) for i in range(-3, 4)]
    energy = ising(image, x, y, horizontal)
    energy += ising(image, x, y, vertical)
    return energy

In [85]:
def energy_diag(image,x,y):
    diag = [(i,i) for i in range(-5, 6)]
    return ising(image, x, y, diag)

In [97]:
def energy_corner(image,x,y):
    adjacent = [(i,j) for i in range(0, 4) for j in range(0, 4)]
    return ising(image, x, y, adjacent)

Funkcja temperatury - wykorzystujemy jedynie eksponencjalną

In [16]:
def img_temp_exp(T, alfa, k):
    return T * alfa

Generujemy obraz 200x200 o $\sigma = 0.4$

In [17]:
img = generate_image(200, 0.4)

Wszystkie gify mają nazwę w postaci 'annealing[nr]_[name].gif'

In [18]:
_, _, _ = optimized_sim_annealing(
    s=img,
    max_iters=300000,
    T=1,
    alfa=0.9997,
    swap_and_update_loss_func=img_swap_and_update_energy,
    swap_func=img_swap_adjacent,
    change_temp_func=img_temp_exp,
    loss_func=energy_ising_16,
    initial_loss_func=calculate_all_energy,
    save=True,
    filename="annealing1_adj_swap.gif",
    duration=40
)

In [19]:
_, _, _ = optimized_sim_annealing(
    s=img,
    max_iters=300000,
    T=1,
    alfa=0.9,
    swap_and_update_loss_func=img_swap_and_update_energy,
    swap_func=img_swap_random,
    change_temp_func=img_temp_exp,
    loss_func=energy_push_and_pull,
    initial_loss_func=calculate_all_energy,
    save=True,
    filename="annealing2_quick_cooling.gif",
    duration=40
)

In [20]:
_, _, _ = optimized_sim_annealing(
    s=img,
    max_iters=300000,
    T=1,
    alfa=0.99995,
    swap_and_update_loss_func=img_swap_and_update_energy,
    swap_func=img_swap_random,
    change_temp_func=img_temp_exp,
    loss_func=energy_push_and_pull,
    initial_loss_func=calculate_all_energy,
    save=True,
    filename="annealing3_slow_cooling.gif",
    duration=40
)

In [21]:
_, _, _ = optimized_sim_annealing(
    s=img,
    max_iters=300000,
    T=1,
    alfa=0.995,
    swap_and_update_loss_func=img_swap_and_update_energy,
    swap_func=img_swap_random,
    change_temp_func=img_temp_exp,
    loss_func=energy_horiz,
    initial_loss_func=calculate_all_energy,
    save=True,
    filename="annealing4_lines.gif",
    duration=40
)

In [39]:
_, _, _ = optimized_sim_annealing(
    s=img,
    max_iters=300000,
    T=1,
    alfa=0.995,
    swap_and_update_loss_func=img_swap_and_update_energy,
    swap_func=img_swap_random,
    change_temp_func=img_temp_exp,
    loss_func=energy_cross,
    initial_loss_func=calculate_all_energy,
    save=True,
    filename="annealing5_three_color.gif",
    duration=40
)

In [80]:
_, _, _ = optimized_sim_annealing(
    s=img,
    max_iters=300000,
    T=1,
    alfa=0.995,
    swap_and_update_loss_func=img_swap_and_update_energy,
    swap_func=img_swap_random,
    change_temp_func=img_temp_exp,
    loss_func=energy_vert_horiz,
    initial_loss_func=calculate_all_energy,
    save=True,
    filename="annealing6_chain.gif",
    duration=40
)

In [84]:
_, _, _ = optimized_sim_annealing(
    s=img,
    max_iters=300000,
    T=1,
    alfa=0.995,
    swap_and_update_loss_func=img_swap_and_update_energy,
    swap_func=img_swap_random,
    change_temp_func=img_temp_exp,
    loss_func=energy_diag,
    initial_loss_func=calculate_all_energy,
    save=True,
    filename="annealing7_diag.gif",
    duration=40
)

In [101]:
_, _, _ = optimized_sim_annealing(
    s=img,
    max_iters=600000,
    T=1,
    alfa=0.995,
    swap_and_update_loss_func=img_swap_and_update_energy,
    swap_func=img_swap_random,
    change_temp_func=img_temp_exp,
    loss_func=energy_corner,
    initial_loss_func=calculate_all_energy,
    save=True,
    filename="annealing8_corner.gif",
    duration=40
)